<a href="https://colab.research.google.com/github/ryanlei309/sdv_cw_final/blob/main/SDV_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [ ]:
# Save the data files as a string
global_data_diversity = "/content/drive/MyDrive/Colab Notebooks/KCLDS_Visulization/nextstrain_ncov_open_global_diversity.tsv"
global_data_metadata = '/content/drive/MyDrive/Colab Notebooks/KCLDS_Visulization/nextstrain_ncov_open_global_metadata.tsv'
uk_data_diversity = '/content/drive/MyDrive/Colab Notebooks/KCLDS_Visulization/nextstrain_groups_ecdc_ncov_united-kingdom_diversity.tsv'
uk_data_metadata = '/content/drive/MyDrive/Colab Notebooks/KCLDS_Visulization/nextstrain_groups_ecdc_ncov_united-kingdom_metadata.tsv'

# Read the data into a dataframe
global_diversity_df = pd.read_csv(global_data_diversity, sep='\t')
global_metadata_df = pd.read_csv(global_data_metadata, sep='\t')
uk_diversity_df = pd.read_csv(uk_data_diversity, sep='\t')
uk_meta_df = pd.read_csv(uk_data_metadata, sep='\t')

# **Data preprocess: Global metadata**

In [ ]:
global_metadata_df.head(3)

,strain,date,S1_mutations,clade_membership,country,current_frequency,division,emerging_lineage,epiweek,genbank_accession,...,mutational_fitness,pango_lineage,recency,region,url,author,accession,originating_lab,submitting_lab,logistic_growth
0,Wuhan-Hu-1/2019,2019-12-26,NaN,19A,China,NaN,China,19A,201952,MN908947,...,NaN,B,Older,Asia,NaN,Wu et al,NaN,NaN,NaN,NaN
1,Wuhan/WH01/2019,2019-12-26,NaN,19A,China,NaN,Hubei,19A,201952,LR757998,...,-0.01,B,Older,Asia,NaN,Hunter et al,NaN,"BGI, Institute of Microbiology, Chinese Academ...",Beijing Genome Institute,NaN
2,AUS/SA02/2020,2020-01-30,NaN,19A,Australia,NaN,South Australia,19A,202005,MT745747,...,0.00,B.1,Older,Oceania,NaN,Caly et al A,NaN,Victorian Infectious Diseases Reference Labora...,The Peter Doherty Institute for Infection and ...,NaN


# **Data preprocessing For research question 1**

In [ ]:
plot1_df = global_metadata_df

# 2019/12~2022/1, total 10 seasons.
# create a clade list, each clade should be 10 times
unique_clade = plot1_df['clade_membership'].unique().tolist()
clade_lst = []
for i in range(len(unique_clade)):
  for j in range(10):
    clade_lst.append(unique_clade[i])

# print(clade_lst)
# create a dictionary, key is clade, value is a dictionary, key is season, value is count.
season_count = {'2019Q4':0, '2020Q1':0, '2020Q2':0, '2020Q3':0, '2020Q4':0
                , '2021Q1':0, '2021Q2':0, '2021Q3':0, '2021Q4':0, '2022Q1':0}

clade_with_count_dict = {}
for i in range(len(unique_clade)):
  clade_with_count_dict[unique_clade[i]] = season_count

# print(clade_with_count_dict)
# Count how many times did the virus appear in each season or each month
# print(type(plot1_df['date'][1]))

# Change the date data type to date 
plot1_df["date"] = pd.to_datetime(plot1_df["date"])

# print(clade_with_count_dict['19A']['2019Q4'])

season_list = []

# Create a column as season
for j in range(len(plot1_df)):
  temp_date = plot1_df.iloc[j]['date']
  temp_clade = plot1_df.iloc[j]['clade_membership']
  
  # When match the clade and certain time, count += 1
  if temp_date.year == 2019 and 10 <= temp_date.month <= 12:
    # clade_with_count_dict[clade]['2019Q4'] += 1
    season_list.append('2019Q4')
  if temp_date.year == 2020 and 1 <= temp_date.month <= 3:
    # clade_with_count_dict[clade]['2020Q1'] += 1
    season_list.append('2020Q1')
  if temp_date.year == 2020 and 4 <= temp_date.month <= 6:
    # clade_with_count_dict[clade]['2020Q2'] += 1
    season_list.append('2020Q2')
  if temp_date.year == 2020 and 7 <= temp_date.month <= 9:
    # clade_with_count_dict[clade]['2020Q3'] += 1
    season_list.append('2020Q3')
  if temp_date.year == 2020 and 10 <= temp_date.month <= 12:
    # clade_with_count_dict[clade]['2020Q4'] += 1
    season_list.append('2020Q4')
  if temp_date.year == 2021 and 1 <= temp_date.month <= 3:
    # clade_with_count_dict[clade]['2021Q1'] += 1
    season_list.append('2021Q1')
  if temp_date.year == 2021 and 4 <= temp_date.month <= 6:
    # clade_with_count_dict[clade]['2021Q2'] += 1
    season_list.append('2021Q2')
  if temp_date.year == 2021 and 7 <= temp_date.month <= 9:
    # clade_with_count_dict[clade]['2021Q3'] += 1
    season_list.append('2021Q3')
  if temp_date.year == 2021 and 10 <= temp_date.month <= 12:
    # clade_with_count_dict[clade]['2021Q4'] += 1
    season_list.append('2021Q4')
  if temp_date.year == 2022 and 1 <= temp_date.month <= 3:
    # clade_with_count_dict[clade]['2022Q1'] += 1
    season_list.append('2022Q1')

season_series = pd.Series(season_list)
plot1_df['season'] = season_series

# Group the dataframe by seaon and clade membership
temp = plot1_df.groupby(['season', 'clade_membership']).size()
temp1 = temp.unstack(level=1)
temp1 = temp1.fillna(0)

# Create a count list by season by each clade
count_list = []
for i in range(len(unique_clade)):
  count = temp1[unique_clade[i]]
  for v in count:
    count_list.append(v)

# Create a season list
season = ['2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1']
season = season*len(unique_clade)

# Combine all the list into a dataframe
final_plot1_df = pd.DataFrame()
final_plot1_df['season'] = season
final_plot1_df['clade'] = clade_lst
final_plot1_df['count'] = count_list


print(final_plot1_df.head(5))
# Save the dataframe as CSV file
final_plot1_df.to_csv('Plot1.csv',index=False)


   season clade  count
0  2019Q4   19A    2.0
1  2020Q1   19A   39.0
2  2020Q2   19A    7.0
3  2020Q3   19A    3.0
4  2020Q4   19A    0.0


# **Data preprocessing for research question 2**

In [ ]:
# Final output should be look like this.
# date,       clade, region,       accumulated region, days
# 12/26/2019  19A    Asia          1                   0
# 1/25/2020   19A    Oceania       2                   29
# 1/29/2020.  19A    North America 3                   33
# 1/30/2020   19A    Europe        4                   34
# 2/28/2020   19A    South America 5                   63

plot2_df = global_metadata_df

# Get unique region
unique_region = plot2_df['region'].unique().tolist()

# Sort the dataframe by date
plot2_df = plot2_df.sort_values(by='date')
plot2_df = plot2_df.reset_index()
# plot2_df.head(8)

# Empty dataframe to store all the clade days information
final_plot2_df = pd.DataFrame()

# Loop over unique clade to 
for i in range(len(unique_clade)):
  # Filter certain clade
  by_clade_df = plot2_df.loc[plot2_df['clade_membership'] == unique_clade[i]]

  # Empty dataframe and lists to store the days count by each clade
  temp_df = pd.DataFrame()
  date = []
  clade = []
  region = []
  accumulate_region = []
  days = []
  acc_region = []

  # Count how many region acccumulated so far
  accumulate_region_count = 0
  # Loop over the clade df
  for j in range(len(by_clade_df)):
    # Extract the whole row
    temp_region = by_clade_df.iloc[j]['region']
    
    # If is a new region, add the date, clade membership, and region information 
    # to a dataframe
    if temp_region not in acc_region:

      # Add the region to control which row to add
      acc_region.append(temp_region)

      # Add date, clade, region and accumulate region to the list
      date.append(by_clade_df.iloc[j]['date'])
      clade.append(by_clade_df.iloc[j]['clade_membership'])
      region.append(by_clade_df.iloc[j]['region'])
      accumulate_region_count += 1
      accumulate_region.append(accumulate_region_count)

  # Add the lists to temp_df
  temp_df['date'] = date
  temp_df['clade'] = clade
  temp_df['region'] = region
  temp_df['accumulate_region'] = accumulate_region

  # Loop over the temp_df to do date different count
  for k in range(len(temp_df)):
    if k == 0:
      days.append(0)
    else:
      first_date = temp_df.iloc[0]['date']
      latest_date = temp_df.iloc[k]['date']
      days_different = (latest_date - first_date).days
      days.append(days_different)
  temp_df['days'] = days
  
  # Combine the dataframe to the final dataframe
  final_plot2_df = final_plot2_df.append(temp_df, ignore_index=True)

# print(final_plot2_df.head(5))

# Save the dataframe as CSV file
final_plot2_df.to_csv('Plot2.csv',index=False)

print(final_plot2_df.to_string())


         date            clade         region  accumulate_region  days
0  2019-12-26              19A           Asia                  1     0
1  2020-01-25              19A        Oceania                  2    30
2  2020-01-29              19A  North America                  3    34
3  2020-01-30              19A         Europe                  4    35
4  2020-02-28              19A  South America                  5    64
5  2020-08-01              19A         Africa                  6   219
6  2020-01-22              19B           Asia                  1     0
7  2020-01-29              19B         Europe                  2     7
8  2020-02-08              19B        Oceania                  3    17
9  2020-03-05              19B  North America                  4    43
10 2020-03-11              19B  South America                  5    49
11 2020-03-30              19B         Africa                  6    68
12 2020-02-04              20A           Asia                  1     0
13 202

# **Data preprocessing for research question 3**

In [ ]:
# Final out put should be like this
# clade duration_days
# 19A.  30
# 19B.  56
# ...   ...

plot3_df = global_metadata_df

# Sort the dataframe by date
plot3_df = plot3_df.sort_values(by='date')
plot3_df = plot3_df.reset_index()

final_plot3_df = pd.DataFrame()
clade = []
duration_days = []

for i in range(len(unique_clade)):
  each_clade_df = plot3_df.loc[plot3_df['clade_membership'] == unique_clade[i]]
  first_date = each_clade_df.iloc[0]['date']

  last_d = len(each_clade_df) - 1
  last_date = each_clade_df.iloc[(len(each_clade_df) - 1)]['date']
  d_day = (last_date - first_date).days

  duration_days.append(d_day)
  clade.append(unique_clade[i])

final_plot3_df['clade'] = clade
final_plot3_df['duration_days'] = duration_days

# Add a week time unit
duration_weeks = []
for i in range(len(final_plot3_df)):
  d_days = final_plot3_df.iloc[i]['duration_days']
  week = round((d_days / 7), 3)
  duration_weeks.append(week)

final_plot3_df['duration_weeks'] = duration_weeks


# Save the dataframe as CSV file
final_plot3_df.to_csv('Plot3.csv',index=False)

print(final_plot3_df.to_string())

              clade  duration_days  duration_weeks
0               19A            687          98.143
1               19B            459          65.571
2               20A            601          85.857
3               20C            415          59.286
4               20G            220          31.429
5        21F (Iota)             34           4.857
6     21C (Epsilon)             94          13.429
7    20H (Beta, V2)            301          43.000
8               20B            648          92.571
9   20J (Gamma, V3)            189          27.000
10              20D            493          70.429
11     21G (Lambda)            196          28.000
12              20F            110          15.714
13  20I (Alpha, V1)            366          52.286
14    21M (Omicron)             40           5.714
15        20E (EU1)            225          32.143
16        21D (Eta)            141          20.143
17      21B (Kappa)              0           0.000
18      21A (Delta)            